In [ ]:
# Install requirements before running this notebook
# !pip install -r requirements.txt

In [ ]:
# Import requested libraries

import psycopg2

import pandas as pd
import numpy as np
import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
# Database parameters

HOST = '***'
PORT = 25060
USERNAME = '***'
PASSWORD =  '***'
DATABASE = 'interview'
#FLAGS : sslmode=require

In [ ]:
# Query PostgreSQL:

try:
    connection = psycopg2.connect(user = USERNAME,
                                  password = PASSWORD,
                                  host = HOST,
                                  port = PORT,
                                  database = DATABASE,
                                  sslmode = 'require')
    # Connection to PostgreSQL
    cursor = connection.cursor()

    # 1st Query Execution: input features table
    query_regressors = 'SELECT * from ccpp.regressors'
    cursor.execute(query_regressors)
    record_reg = cursor.fetchall()
    cur_desc = cursor.description
    column_names=[]
    for col in cur_desc:
        column_names.append(col.name)
    
    # 2nd Query Execution: target table
    query_target = 'SELECT * from ccpp.target'
    cursor.execute(query_target)
    record_target = cursor.fetchall()
       
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    # Closing database connection
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [ ]:
column_names

In [ ]:
X = pd.DataFrame(record_reg, columns=['ID','AT','V','AP','RH'])
print(X.head())
print(X.shape)
X.describe()

In [ ]:
Y = pd.DataFrame(record_target, columns=['ID','PE'])
print(Y.head())
print(Y.shape)
Y.describe()

In [ ]:
df = pd.merge(X, Y, on='ID', how='inner')
df.shape

In [ ]:
# Check intersection of IDs
assert len(list(set(X.ID) & set(Y.ID)))==df.shape[0]

Use pandas profiling for a first deep dive into data:

In [ ]:
profile_reg = pandas_profiling.ProfileReport(X)
profile_reg.to_file(outputfile="input_regressor_report.html")

profile_tgt = pandas_profiling.ProfileReport(Y)
profile_tgt.to_file(outputfile="target_report.html")

profile = pandas_profiling.ProfileReport(df)
profile.to_file(outputfile="complete_dataset_report.html")

In [ ]:
profile

From analysis in report:
* from the Spearman correlation heatmap we can see that **V and AT are strongly correlated to each other**
* from the same heatmap we can see that **AT is strongly and negativly correlated to target PE**. So does V.
* Distribution of AP and PE need to be investigated: **possible outliers**

In [ ]:
input_features = ['AT', 'V', 'AP', 'RH']

## Deal with missing values
From report analysis: 
* one missing value in target --> remove row
* one missing value in AT --> Fill element (scikit learn provides SimpleImputer or IterativeImputer -beta-) 

In [ ]:
# Drop row with missing value in target variable
print(df[np.isnan(df.PE)])
df.drop(df[np.isnan(df.PE)].index[0], axis=0, inplace=True)


In [ ]:
# Check that AT is the only field with nan values
df.columns[df.isnull().any()]

In [ ]:
# Fill na values with SimpleImputer. Best is IterativeImputer but it is in beta release
df_columns=df.columns
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df_array = imp.fit_transform(df)  
df = pd.DataFrame(df_array, columns=df_columns)

## Outliers

Remove oultiers from each field. 

From analysis in pandas profiling, let's investigate AP first.

In [ ]:
# Distribution plot  and boxplot of AP
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(8,3))
sns.distplot(df.AP, ax=axes[0])
sns.boxplot(df.AP, ax=axes[1])
plt.show()

In [ ]:
# Remove the row with outliers in dataframe
print(df[df['AP']>=10000])
df.drop(df[df['AP']>=10000].index[0],axis=0, inplace=True)

In [ ]:
# Check distribution plot  and boxplot of AP. Are there more outliers? NO
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(8,3))
sns.distplot(df.AP, ax=axes[0])
sns.boxplot(df.AP, ax=axes[1])
plt.show()


Check distribution plots to find possible outliers for the remaining fields

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(16,4))
for ax, col in zip(axes, input_features):
    sns.distplot(df[col], ax=ax)

plt.show()

And what about target variable?

In [ ]:
plt.figure(figsize=(4, 3))
sns.boxplot(df.PE)

In [ ]:
# Target variable has one outlier: remove it from the dataframe
df.drop(df[df.PE>400000].index[0], axis=0, inplace=True)

In [ ]:
plt.figure(figsize=(4, 3))
sns.distplot(df.PE)

In [ ]:
# Let's see trends and correlations in the pairgrid plots

In [ ]:
g = sns.PairGrid(df[input_features+['PE']])
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)

We can easily see how AT and the target variable PE are strongly correlated. We can suppose that AT is one of the most important variables in predicting PE. Let's check it with some regression models.

## Standardization

In [ ]:
# Standardize all features before applying regression models, specially for linera regression

In [ ]:
target = df.PE
scaler = StandardScaler()
scaler.fit(df[input_features])
print(scaler.mean_)
X_input=pd.DataFrame(scaler.transform(df[input_features]), columns=input_features)


## Models and variable importance

In [ ]:
# Import libraries from scikit-learn. Let's first test some classical machine learning models.
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost

In [ ]:
# Linear regression model, evaluated with 10-fold cross validation
CV=10
reg = LinearRegression().fit(X_input, target)
np.mean(cross_val_score(reg, X_input, target, scoring='r2',cv=CV))

Get variable importance from coefficients set by the regression model for each input feature


In [ ]:
reg_coeff = [(x,y) for x,y in zip(input_features,reg.coef_)]
reg_coeff

The regression model confirms that AT is the most important feature for the PE prediction. Its importance was already known by the Spearman correlation coefficient between AT and Target PE.

Let's compare more models in order to estimate the best performance. (GridSearchCV should be applied to check different hyperparameters configurations)

In [ ]:
sgd = SGDRegressor(max_iter=1000, tol=1e-3)
sgd.fit(X_input, target)
print("Mean R2 score over {}-folds {}".format(CV,np.mean(np.mean(cross_val_score(sgd, X_input, target, scoring='r2',cv=CV)))))
sgd_coeff = [(x,y) for x,y in zip(input_features,sgd.coef_)]
sgd_coeff

In [ ]:
rfr = RandomForestRegressor(max_depth=8, random_state=42,  n_estimators=100)
rfr.fit(X_input,target)
print("Mean R2 score over {}-folds {}".format(CV,np.mean(cross_val_score(rfr, X_input, target, scoring='r2',cv=CV))))
rfr_feature_imp = [(x,y) for x,y in zip(input_features,rfr.feature_importances_)]
rfr_feature_imp

In [ ]:
%%capture
xgb = xgboost.XGBRegressor(max_depth=8, n_estimators=100, booster='gbtree', random_state=42, verbosity=0)
#xgb.fit(X_input,target)
mean_xgb_score=np.mean(np.mean(cross_val_score(xgb, X_input, target, scoring='r2',cv=CV)))

In [ ]:
print("Mean R2 score over {}-folds {}".format(CV,mean_xgb_score))
xgb_feature_imp = [(x,y) for x,y in zip(input_features,xgb.feature_importances_)]
xgb_feature_imp

XGBoost is the model with the best performance on the provided dataset. The feature importance got from this model confirms the relevance of AT in predicting PE.